到目前为止，我们介绍了如何处理数据以及如何构建、训练和测试深度学习模型。然而在实际中，我们有时需要把训练好的模型部署到很多不同的设备。在这种情况下，我们可以**把内存中训练好的模型参数存储在硬盘上**供后续读取使用。

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.3.0


## 4.5.1 load and save NDarray  保存NDarray

我们可以直接使用`save`函数和`load`函数分别**存储**和**读取**。下面的例子创建了tensor`x`，并将其存在文件名同为`x`的文件里。

In [2]:
import numpy as np

x = tf.ones(3)
x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>

然后我们将数据从存储的文件读回内存。

In [4]:
np.save('x.npy', x)
x2 = np.load('x.npy')   # npy文件，原来是这样来的
x2

array([1., 1., 1.], dtype=float32)

我们还可以存储一列`tensor`并读回内存。

In [5]:
y = tf.zeros(4)
y

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

In [7]:
np.save('xy.npy',[x,y])   # 这里的格式，需要学习下
x2, y2 = np.load('xy.npy', allow_pickle=True)
(x2, y2)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

我们甚至可以存储并读取一个从字符串映射到`tensor`的字典。

In [8]:
mydict = {'x':x, 'y':y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

## 4.5.2 load and save model parameters  保存模型参数

我们还可以读写模型的参数。
为了演示方便，我们先创建一个多层感知机，并将其初始化。。

In [9]:
x = tf.random.normal((2,20))
x

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[-7.3711472e-03, -2.2265685e+00,  1.6618804e+00,  2.5140259e-01,
        -7.2410059e-01, -1.0122397e+00,  2.5616276e-01, -4.1212311e-01,
         4.8814896e-01,  9.1878396e-01, -2.7675667e-01, -1.0136944e+00,
         1.2225481e+00,  6.1943024e-01,  2.0574312e-01,  9.0819165e-02,
         3.4139338e-01, -1.3885272e-01,  1.4656541e+00,  4.3042353e-01],
       [ 5.9879646e-03, -3.8827896e-01,  3.9519036e-01,  5.6982005e-01,
         7.1759450e-01, -8.6360025e-01, -6.9179755e-01, -2.3116879e+00,
         1.0306482e+00, -4.5546532e-02,  2.1602666e-01,  7.3800533e-04,
         1.9990959e+00,  1.8492056e+00,  1.6333201e+00,  2.9170945e+00,
        -1.6078517e+00, -1.0648005e+00, -8.0883843e-01,  2.1114954e-01]],
      dtype=float32)>

In [11]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output

net = MLP()
Y = net(x)
Y

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.28522736, -0.41203678,  0.22189781,  0.15751928, -0.22163066,
        -0.16435435, -0.33152458, -0.08982636, -0.339939  , -0.1964671 ],
       [ 0.793442  , -0.57389915, -0.75346386,  0.11127923, -0.15200633,
        -0.2400654 , -0.74082786, -0.01516237, -0.17271438,  0.25783202]],
      dtype=float32)>

下面把该模型的参数存成文件，文件名为`4.5saved_model.h5`。

In [12]:
net.save_weights("4.5saved_model.h5")   # 记住这里的表述方式

接下来，我们再实例化一次定义好的多层感知机。与随机初始化模型参数不同，我们在这里直接读取保存在文件里的参数。

因为这两个实例都有同样的模型参数，那么对同一个输入X的计算结果将会是一样的。我们来验证一下。

In [14]:
net2 = MLP()
net2(x)
net2.load_weights("4.5saved_model.h5")
Y2 = net2(x)
Y2 == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

> 注：本节除了代码之外与原书基本相同，[原书传送门](https://zh.d2l.ai/chapter_deep-learning-computation/read-write.html)

